# Data Preprocessing

## 1. Introduction

In this notebook, we will preprocess the data to make it ready for the model. We will focus on data cleaning, integration, and aggregation. We will discuss methodologies investigated and chosen for handling of missing values, feature engineering, and feature selection.

## 2. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

# Load the environment variables
load_dotenv("../config/.env")

scripts_path = os.getenv("SCRIPTS_PATH")

# Add the path to the scripts folder and import the functions
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

In [ ]:
# from raw_data import get_raw_dataframes function
from raw_data import get_raw_dataframes

## 3. Load Data

In [ ]:
# Load data
city_data, country_data, weather_data, migraine_data = get_raw_dataframes()

# Check the shape of the dataframes
city_data.shape, country_data.shape, weather_data.shape, migraine_data.shape

## 4. Data Cleaning

### 4.1 Drop Unnecessary Columns/Rows

#### 4.1.1 DataFrame: `city_data`

In [ ]:
# View the `city_data` dataframe
city_data

Keeping all columns for now.

#### 4.1.2 DataFrame: `country_data`

In [ ]:
# View the `country_data` dataframe
country_data

*Keeping* the following columns:
- 'country'
- 'iso2'
- 'iso3'

*Removing* the following columns:
- 'native_name'
- 'population'
- 'area'
- 'capital'
- 'capital_lat'
- 'capital_lng'
- 'region'
- 'continent'

In [ ]:
# Drop columns that are not needed for the analysis
country_data.drop(columns=['native_name', 'population', 'area', 'capital', 'capital_lat', 'capital_lng', 'region', 'continent'], inplace=True)

# Check the shape of the dataframe
country_data.shape

#### 4.1.3 DataFrame: `weather_data`

In [ ]:
# View the `weather_data` dataframe
weather_data

*Keeping* all columns:
- 'station_id'
- 'city_name'
- 'date'
- 'season'
- '*_temp_c' (avg, min, max)
- 'precipitation_mm'
- 'snow_depth_mm'
- 'avg_wind_dir_deg'
- 'avg_wind_speed_kmh'
- 'peak_wind_gust_kmh'
- 'avg_sea_level_pres_hpa'
- 'sunshine_total_min'

#### 4.1.4 DataFrame: `migraine_data`

In [ ]:
# View the migraine data
migraine_data

*Keeping* all columns:
- 'measure'
- 'location'
- 'sex'
- 'age'
- 'cause'
- 'metric'
- 'year'
- 'val'
- 'upper'
- 'lower'

In [ ]:
# Check the unique values of the measure_name column
migraine_data['measure'].unique()

In [ ]:
# Check the unique values of the metric_name column
migraine_data['metric'].unique()

For this analysis, we will only be looking at the prevalence (total # of cases in the population) of headache disorders (i.e., migraine, tension-type headache) and confirmed no other measure name columns are present in our dataset. We will remove all percent and rate values, as we are only interested in the total number of cases.

In [ ]:
# Filter the data to only include the number of headache and migraine cases

# Identify indices to drop for both 'metric'
metric_indices_to_drop = migraine_data[migraine_data['metric'].isin(['Percent', 'Rate'])].index

# Drop rows
filtered_migraine_data = migraine_data.drop(metric_indices_to_drop)

# Format values in the val, upper, and lower columns to two decimal places
pd.set_option('display.float_format', lambda x:'%.2f' % x)

filtered_migraine_data

### 4.2 Review and Plan for Missing/Zero Values

#### 4.2.1 DataFrame: `city_data`

In [ ]:
# Check for missing values
print("\nCity Missing Values:\n")
print(city_data.isnull().sum())

# Calculate zero counts for each column
print("\nCity Zero Counts:\n")
zero_counts = (city_data == 0).sum()
print(zero_counts)

Plan: Merge with `country_data` then merge with `weather_data` to fill in missing values.

#### 4.2.2 DataFrame: `country_data`

In [ ]:
# Check for missing values
print("\nCountry Missing Values:\n")
print(country_data.isnull().sum())

# Calculate zero counts for each column
print("\nCountry Zero Counts:\n")
zero_counts = (country_data == 0).sum()
print(zero_counts)

Plan: Merge with `city_data` then merge with `weather_data` to fill in missing values.

#### 4.2.3 DataFrame: `weather_data`

In [ ]:
# Check for missing values
print("\nWeather Missing Values:\n")
print(weather_data.isnull().sum())

# Calculate zero counts for each column
print("\nWeather Zero Counts:\n")
zero_counts = (weather_data == 0).sum()
print(zero_counts)

Plan: Merge with `city_data` and `country_data` to fill in missing values. Then recheck missing and zero counts after merging.

#### 4.2.4 DataFrame: `filtered_migraine_data`

In [ ]:
# Check for missing values
print("\nFiltered Migraine Missing Values:\n")
print(filtered_migraine_data.isnull().sum())

# Check for zero values
print("\nFiltered Migraine Zero Counts:\n")
zero_counts = (filtered_migraine_data == 0).sum()
print(zero_counts)

No missing values in this dataset.

Plan: Merge with combined and filtered weather data.

We need to address the zero values in the `val`, `upper`, and `lower` columns. We will investigate the distribution of these values to determine the best method for handling them.

In [ ]:
# Print rows with zero values
zero_rows_any = filtered_migraine_data[(filtered_migraine_data == 0).any(axis=1)]
print(zero_rows_any)

In [ ]:
# Check the counts of the population's age groups
filtered_migraine_data['age'].value_counts()

The number of zero values in the `val`, `upper`, and `lower` columns is 6,100. After further investigation, there are also 6,100 rows where the age of the population is <5 years old. Since this is perfectly reasonable explanation, we will remove these rows from the dataset.

In [ ]:
# Drop rows that meet the condition
filtered_migraine_data.drop(
    filtered_migraine_data.query("`age` == '<5 years' and `val` == 0").index, 
    inplace=True
)

In [ ]:
zero_rows_any = filtered_migraine_data[(filtered_migraine_data == 0).any(axis=1)]
print(zero_rows_any)

In [ ]:
filtered_migraine_data['age'].value_counts()

In [ ]:
filtered_migraine_data.shape

### 4.3 Standardize Country and State Names across datasets

#### 4.3.1 DataFrame: `city_data`

##### 4.3.1.1 Country Names

In [ ]:
# Import the function
from data_location_matcher import find_matching_and_non_matching

# Find matching and non-matching countries
city_data_matching_countries, city_data_non_matching_countries = find_matching_and_non_matching(city_data, 'country')

# View the matching countries
city_data_matching_countries

# View the non-matching countries
city_data_non_matching_countries

In [ ]:
city_data_country_replacement_dict = { 
    'Guinea Bissau': 'Guinea-Bissau',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'Macau S.A.R': 'Macau',
    'Svalbard and Jan Mayen Islands': 'Svalbard and Jan Mayen',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the city dataframe
city_data['country'].replace(city_data_country_replacement_dict, inplace=True)

city_data['country'].unique()

##### 4.3.1.2 State Names

In [ ]:
# Find matching and non-matching states
city_data_matching_states, city_data_non_matching_states = find_matching_and_non_matching(city_data, 'state')

# View the matching states
city_data_matching_states

# View the non-matching states
city_data_non_matching_states

#### 4.3.2 DataFrame: `country_data`

##### 4.3.2.1 Country Names

In [ ]:
# Find matching and non-matching countries
country_data_matching_countries, country_data_non_matching_countries = find_matching_and_non_matching(country_data, 'country')

# View the matching countries
country_data_matching_countries

# View the non-matching countries
country_data_non_matching_countries

In [ ]:
country_data_country_replacement_dict = {
    'Democratic Republic of the Congo': 'Congo (Kinshasa)',
    'Republic of the Congo': 'Congo (Brazzaville)',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'São Tomé and Príncipe': 'Sao Tome and Principe', 
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the country dataframe
country_data['country'].replace(country_data_country_replacement_dict, inplace=True)

country_data['country'].unique()

##### 4.3.2.2 State Names

No state names in this dataset.

#### 4.3.3 DataFrame: `migraine_data`

##### 4.3.3.1 Country Names

In [ ]:
filtered_migraine_data.head()

In [ ]:
from data_location_matcher import COUNTRIES, US_STATES

migraine_data_countries_and_states = filtered_migraine_data['location'].unique()

migraine_data_countries_and_states

In [ ]:
migraine_data_location_replacement_dict = {
    'Taiwan (Province of China)': 'Taiwan',
    'Viet Nam': 'Vietnam',
    "Democratic People's Republic of Korea": 'North Korea',
    "Lao People's Democratic Republic": 'Laos',
    'Democratic Republic of the Congo': 'Congo (Kinshasa)',
    'Micronesia (Federated States of)': 'Micronesia',
    'North Macedonia': 'Macedonia',
    'Brunei Darussalam': 'Brunei',
    'Republic of Korea': 'South Korea',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Iran (Islamic Republic of)': 'Iran',
    'United Republic of Tanzania': 'Tanzania',    
    'Republic of the Congo': 'Congo (Brazzaville)',
    'Republic of Moldova': 'Moldova',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'São Tomé and Príncipe': 'Sao Tome and Principe', 
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the country dataframe
filtered_migraine_data['location'].replace(migraine_data_location_replacement_dict, inplace=True)

filtered_migraine_data['location'].unique()

In [ ]:
# Convert your original list and the list of U.S. states to sets
set_migraine_countries_and_states = set(migraine_data_countries_and_states)
set_US_states = set(US_STATES)
set_countries = set(COUNTRIES)

# Create a new list excluding the U.S. states
migrained_filtered_countries_list = [item for item in set_migraine_countries_and_states if item not in set_US_states]

# View the list
migrained_filtered_countries_list

##### 4.3.3.2 State Names

In [ ]:
# Convert the list of U.S. states to sets
set_US_states = set(US_STATES)

# Create a new list including the U.S. states
migrained_filtered_states_list = [item for item in set_migraine_countries_and_states if item in set_US_states]

# View the list
migrained_filtered_states_list

## 5. Data Integration

### 5.1 Overview

Briefly introduce the goal of data integration in the context of this project. Provide a high-level view of the datasets that will be integrated.

### 5.2 Data Sources

#### 5.2.1 Weather Data

The weather data provides context regarding sea level pressure, sunshine, temperature, and precipitation for each city. This data is relevant because it provides information about the weather conditions that may be associated with migraine prevalence. The country data will be combined with the city data to provide additional information about each city, such as the country, region, and continent. The combined city and country data will then be combined with the weather data to provide additional information.

The daily weather data source file is quite large and is provided in a .parquet format for low memory consumption and data type preservation. 

##### 5.2.1.1 Cities DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | cities.csv                                         |
| **Data Source Format** | CSV (comma-separated values)                       |
| **Data Source Desc** | Individual cities and weather stations around the world |
| **Data Source Size** | 84.1 KB                                             |
|                      | 1,245 rows                                          |
|                      | 8 columns                                           |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                          |

**Data Source Columns**

| Column Name  | Description                               |
|--------------|-------------------------------------------|
| `station_id` | Unique ID for the weather station.        |
| `city_name`  | Name of the city.                         |
| `country`    | The country where the city is located.    |
| `state`      | The state or province within the country. |
| `iso2`       | The two-letter country code.              |
| `iso3`       | The three-letter country code.            |
| `latitude`   | Latitude coordinate of the city.          |
| `longitude`  | Longitude coordinate of the city.         |


##### 5.2.1.2 Countries DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | countries.csv                                         |
| **Data Source Format** | CSV (comma-separated values)                       |
| **Data Source Desc** | Individual country geographic and demographic characteristics |
| **Data Source Size** | 20.6 KB                        |
|                      | 214 rows                                    |
|                      | 11 columns                                       |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                           |

**Data Source Columns**

| Column Name  | Description                                               |
|--------------|-----------------------------------------------------------|
| `iso3`       | The three-letter code representing the country.           |
| `country`    | The English name of the country.                          |
| `native_name`| The native name of the country.                           |
| `iso2`       | The two-letter code representing the country.             |
| `population` | The population of the country.                            |
| `area`       | The total land area of the country in square kilometers.  |
| `capital`    | The name of the capital city.                             |
| `capital_lat`| The latitude coordinate of the capital city.              |
| `capital_lng`| The longitude coordinate of the capital city.             |
| `region`     | The specific region within the continent where the country is located. |
| `continent`  | The continent to which the country belongs.               |

##### 5.2.1.3 Daily Weather DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | daily_weather.parquet                              |
| **Data Source Format** | .parquet (compressed, maintains original data types, efficient)|
| **Data Source Desc** | Daily weather data                            |
| **Data Source Size** | 233 MB                                        |
|                      | 27,635,763 rows                               |
|                      | 14 columns                                    |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                           |

**Data Source Columns**

| Column Name            | Description                                       |
|------------------------|---------------------------------------------------|
| `station_id`           | Unique ID for the weather station.                |
| `city_name`            | Name of the city where the station is located.    |
| `date`                 | Date of the weather record.                       |
| `season`               | Season corresponding to the date (e.g., summer, winter).|
| `avg_temp_c`           | Average temperature in Celsius.                   |
| `min_temp_c`           | Minimum temperature in Celsius.                   |
| `max_temp_c`           | Maximum temperature in Celsius.                   |
| `precipitation_mm`     | Precipitation in millimeters.                     |
| `snow_depth_mm`        | Snow depth in millimeters.                        |
| `avg_wind_dir_deg`     | Average wind direction in degrees.                |
| `avg_wind_speed_kmh`   | Average wind speed in kilometers per hour.        |
| `peak_wind_gust_kmh`   | Peak wind gust in kilometers per hour.            |
| `avg_sea_level_pres_hpa`| Average sea-level pressure in hectopascals.      |
| `sunshine_total_min`   | Total sunshine duration in minutes.               |

#### 5.2.2 Migraine Data

The migraine data provides information about the prevalence of migraine in different countries. This data is relevant because it provides information about the prevalence of migraine by gender, age, year, and location. This data will be combined with the weather data to determine if there is a relationship between weather and migraine prevalence.

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | IHME-GBD_2019_DATA-2c1d3941-1.csv                  |
|                      | IHME-GBD_2019_DATA-2c1d3941-2.csv                  |
|                      | IHME-GBD_2019_DATA-2c1d3941-3.csv                  |
| **Data Source Format** | CSV (comma-separated values)                     |
| **Data Source Desc** | All GBD causes, risks, impairments, etiologies, and injuries by nature |
| **Data Source Size** | 158 MB                                              |
|                      | 1,377,000 rows                                      |
|                      | 10 columns                                          |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                          |

**Data Source Columns**

| Column Name    | Description                                          |
|----------------|------------------------------------------------------|
| `measure` | The name of measure.                                      |
| `location`| The name of each location.                                |
| `sex`     | The name of each sex choice.                              |
| `age`     | The name of each age group.                               |
| `cause`   | The name of each cause.                                   |
| `metric`  | The name of each metric/unit.                             |
| `year`    | The annual results for all measures.                      |
| `val`     | The value of each metric/unit.                            |
| `upper`   | The 95% Uncertainty Interval - Upper Bound value.         |
| `lower`   | The 95% Uncertainty Interval - Lower Bound value.         |

### 5.3 Preliminary Steps

**Overview**

In this section, the focus is on preparing the dataset for further analysis and exploration. The steps include merging multiple data sources, filtering the data based on specific criteria, cleaning the data by dropping unnecessary columns and rows, and conducting a preliminary analysis through correlation metrics. Each of these steps is essential for ensuring the data's integrity, usability, and relevance to the study objectives.

**5.3.1 Data Merging**

The first step involves merging the city and country datasets using a common identifier. This integration provides a comprehensive view that combines geographical and political attributes. Following that, the weather dataset is integrated with the already combined city-country data. The resulting dataset offers a rich context, incorporating both geographical information and meteorological variables.

**5.3.2 Data Filtering**

The dataset is filtered to only include records pertaining to US cities, thereby narrowing the scope for more targeted analysis. Further filtering is done to include only specific years, enhancing the dataset's relevance to the study period.

**5.3.3 Data Cleaning**

Columns that do not contribute to the analysis or contain redundant information are dropped to simplify the dataset. Rows with missing or irrelevant data are removed to improve the dataset's quality and consistency. Duplicate rows, if any, are identified and removed to ensure each record in the dataset is unique.

**5.3.4 Preliminary Analysis**

A correlation analysis is conducted on specific weather attributes like temperature, precipitation, and wind speed to identify any significant relationships among them.

---

Throughout these steps, the data are continuously inspected to understand their structures, types, and quality. Various data profiling techniques are employed, such as examining data distributions, checking for missing values, and assessing data types, to ensure that the dataset meets the quality and integrity requirements for downstream analysis.

#### 5.3.1 Data Merging

##### 5.3.1.1 Merge `city_data` and `country_data`

Join the countries and cities tables on the `country`, `iso2`, and `iso3` columns to give more context to the weather data.

In [ ]:
# Code for joining countries and cities
city_country = city_data.merge(country_data, 
                               how='left', 
                               left_on=['country', 'iso2', 'iso3'], 
                               right_on=['country', 'iso2', 'iso3']
                               )

# Review the shape of the new dataframe
city_country

##### 5.3.1.2 Merge `weather_data` and `city_country`

Join the weather data with the combined countries and cities tables on the `station_id` and `city_name` columns.

In [ ]:
# Review the shape of the weather dataframe
print(f"Weather Data: {weather_data.shape}")

# Review the shape of the city-country dataframe
print(f"City-Country Data: {city_country.shape}")

In [ ]:
weather_data.head(1)

In [ ]:
# Combine city/country with daily weather data
combined_weather = weather_data.merge(city_country, 
                                      how='left', 
                                      left_on=['station_id', 'city_name'], 
                                      right_on=['station_id', 'city_name']
)

# Review the shape of the new dataframe
print(f"Combined Weather Data: {combined_weather.shape}")

In [ ]:
combined_weather.head(1)

In [ ]:
combined_weather['country'].unique()

In [ ]:
# View the states where country is 'United States of America'
combined_weather[combined_weather['country'] == 'United States of America']['state'].unique()

In [ ]:
combined_weather.isnull().sum()

In [ ]:
combined_weather.shape

##### 5.3.1.3 Merge `filtered_migraine_data` and `combined_weather_data`

Before we can merge the migraine data with the weather data, we need to standardize the `location` column in the migraine data. First, we will split the `location` column into `country` and `state` columns. We have already confirmed that the `city_country` dataframe contains both a country and state column so we will use this dataframe to populate the `country` and `state` columns in the migraine data. We will then drop the `location` column from the migraine data. Finally, we will merge the migraine data with the weather data on the `country`, `state`, and `year` columns.

In [ ]:
filtered_migraine_data.head(1)

In [ ]:
# import the assign_country function
from cleanup_location_migraine import assign_country

# Assign country to migraine data
filtered_migraine_data['country'] = filtered_migraine_data['location'].apply(assign_country, args=(city_country,))

filtered_migraine_data.head(1)

In [ ]:
# import the assign_state function
from cleanup_location_migraine import assign_state

# Assign state to migraine data
filtered_migraine_data['state'] = filtered_migraine_data['location'].apply(assign_state, args=(city_country,))

filtered_migraine_data.head(1)

In [ ]:
filtered_migraine_data.isnull().sum()

In [ ]:
filtered_migraine_data.shape

In [ ]:
# Combine filtered migraine data with combined weather data
weather_migraine_2 = combined_weather.merge(filtered_migraine_data, 
                                      how='left', 
                                      left_on=['country'], 
                                      right_on=['country']
)

# Review the shape of the new dataframe
print(f"Combined Weather and Migraine Data: {weather_migraine_2.shape}")

weather_migraine_2.head(1)

In [ ]:
weather_migraine_2.isnull().sum()

#### 5.3.2 Data Filtering

##### 5.3.2.1 Filter `combined_weather` by US Cities

In [ ]:
# Filter the combined weather data to only include the US
usa_weather = combined_weather[combined_weather['iso3'] == 'USA']

# Review the shape of the new dataframe
usa_weather.shape

In [ ]:
# View 10 rows of the new dataframe
usa_weather

In [ ]:
# Check for missing values
usa_weather.isnull().sum()

In [ ]:
# Check the unique values of the iso3 column, confirming no other countries are included
usa_weather['iso3'].unique()

##### 5.3.2.2 Filter `usa_weather` by Year

Given that the migraine data is annual, we need to add a 'year' column to the weather data and filter it by year.

In [ ]:
# confirmed that date column is in datetime format
usa_weather.dtypes

In [ ]:
usa_weather = usa_weather.copy()

# Add a column for the year using .loc, specifying to add a new column for all rows
usa_weather['year'] = usa_weather['date'].dt.year

# Add a column for the month using .loc, specifying to add a new column for all rows
usa_weather['month'] = usa_weather['date'].dt.month

usa_weather.head()

In [ ]:
usa_weather['year'].unique()

In [ ]:
usa_weather['year'].describe()

Earliest year is 1872, latest year is 2023. We will filter the weather data to only include years 1990-2019 to match the migraine data's date range.

In [ ]:
usa_weather.shape

In [ ]:
# Filter the data to only include the years 1990-2019
year_filter = (usa_weather['year'] >= 1990) & (usa_weather['year'] <= 2019)
usa_weather = usa_weather[year_filter]

# Review the shape of the new dataframe
usa_weather.shape

In [ ]:
usa_weather['year'].unique()

In [ ]:
usa_weather.isnull().sum()

#### 5.3.3 Data Cleaning (2nd Round)

##### 5.3.3.1 Drop Unnecessary Columns

After further review of the data, the `country` and `iso2` columns are no longer needed since we have filtered for iso3=USA, so we will drop them.

In [ ]:
# List of columns to keep
columns_to_keep = [col for col in usa_weather.columns if col not in ['country', 'iso2']]

# Use .loc to select only the columns to keep
usa_weather = usa_weather.loc[:, columns_to_keep]

In [ ]:
# Review the shape of the new dataframe
usa_weather.shape

##### 5.3.3.2 Drop Unnecessary Rows

In [ ]:
usa_weather.isnull().sum()

In [ ]:
# Drop rows with missing values
usa_weather = usa_weather.dropna(subset=['min_temp_c', 'max_temp_c', 'precipitation_mm'])

# Review the shape of the new dataframe
usa_weather.shape

In [ ]:
# Check for missing values
usa_weather.isnull().sum()

##### 5.3.3.3 Drop Duplicate Rows

In [ ]:
# Original DataFrame
usa_weather_row_count = len(usa_weather)

# DataFrame after dropping duplicates
usa_weather_deduplicated = usa_weather.drop_duplicates()
deduplicated_row_count = len(usa_weather_deduplicated)

# Calculate the number of rows that would be dropped
rows_to_be_dropped = usa_weather_row_count - deduplicated_row_count

# Print the difference
print(f"Rows to be dropped: {rows_to_be_dropped}")

#### 5.3.4 Preliminary Analysis

##### 5.3.4.1 Correlation Analysis for Weather Features

In [ ]:
weather_features = ['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm', 'snow_depth_mm', 'avg_wind_dir_deg',\
                     'avg_wind_speed_kmh', 'peak_wind_gust_kmh', 'avg_sea_level_pres_hpa', 'sunshine_total_min']
correlation_matrix_usa_weather = usa_weather[weather_features].corr()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_usa_weather, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.show()

The `sunshine_total_min` column has a lot of missing values, has a very weak correlation (-0.0066) with `avg_sea_level_pres_hpa`, and is not a focal point of this analysis, so we will drop that column.

In [ ]:
# Columns to keep
columns_to_keep = [col for col in usa_weather.columns if col not in ['sunshine_total_min']]

# Use .loc to select only the columns to keep
usa_weather = usa_weather.loc[:, columns_to_keep]

# Check for missing values
usa_weather.isnull().sum()

### 5.4 Handling Missing Values

#### 5.4.1 Non-pressure-related Columns

##### 5.4.1.1 Average Temperature Interpolation and Validation

For the `avg_temp_c` missing values, we will calculate the average of the `min_temp_c` and `max_temp_c` columns and use that value to fill in the missing average temperature values. A new, temporary column will be created called `avg_temp_c_interpolated` to hold these values.

In [ ]:
# Create a copy of the dataframe
usa_weather = usa_weather.copy()

# Create a column for calculating the `avg_temp_c` using the `min_temp_c` and `max_temp_c` columns
usa_weather['avg_temp_c_interpolated'] = usa_weather['avg_temp_c'].combine_first\
    ((usa_weather['min_temp_c'] + usa_weather['max_temp_c']) / 2)

Utilize the mean absolute error (MAE) to determine the accuracy of the interpolated values.

In [ ]:
# Import mean_absolute_error from sklearn.metrics
from sklearn.metrics import mean_absolute_error

# Filter out rows where either of the two columns is NaN
filtered_df = usa_weather.dropna(subset=['avg_temp_c', 'avg_temp_c_interpolated'])

# Calculate mean absolute error
mae = mean_absolute_error(filtered_df['avg_temp_c'], filtered_df['avg_temp_c_interpolated'])

# Print the mean absolute error
print(f"Mean Absolute Error: {mae}")


As noted above, the MAE is 0.0, so we will use the interpolated values to fill in the missing values for the `avg_temp_c` column. We will then drop the `avg_temp_c_interpolated` column. Please note after further investigation, it was found that the original `avg_temp_c` values are precisely calculated as the average of `min_temp_c` and `max_temp_c` values, so there is no loss of information.

Replace the missing values in the `avg_temp_c` column with the values from the `avg_temp_c_interpolated` column.

In [ ]:
# Fill missing values in the 'avg_temp_c' column with the average of the 
# 'min_temp_c' and 'max_temp_c' columns
usa_weather['avg_temp_c'] = usa_weather['avg_temp_c_interpolated']

Drop the `avg_temp_c_interpolated` column and check for any remaining missing values.

In [ ]:
# Drop the 'avg_temp_c_interpolated' column
usa_weather.drop(columns=['avg_temp_c_interpolated'], inplace=True)

# Check for missing values
usa_weather.isnull().sum()

In [ ]:
# Print the shape of the dataframe
print(f"Original Shape: {usa_weather.shape}")

##### 5.4.1.2 Aggregate Weather Data by Year and State

The migraine data is aggregated at an annual level and broken down by state, so we need to aggregate the weather data to match. A mean aggregation will be used for all columns except for the `precipitation_mm` and `snow_depth_mm` columns, which will be aggregated using a sum.

1. **Group by Year, State, and City Name**: Use pandas' `groupby` method to group data by both the `year`, `state`, and `city` columns.
2. **Aggregation Functions**: 
    - For temperatures (`avg_temp_c`, `min_temp_c`, `max_temp_c`), the mean is calculated for each year and state.
    - For wind (`avg_wind_dir_deg`, `avg_wind_speed_kmh`, `peak_wind_gust_kmh`), the mean is calculated for each year and state.
    - For `precipitation_mm` and `snow_depth_mm`, the total sum is calculated for each year and state.
    - For `avg_sea_level_pres_hpa`, the mean is calculated, assuming it's relevant to have an annual mean sea level pressure for each state.
3. **Spatial Data**: For latitude and longitude, the first observed value for each year and state is taken, assuming that these values are consistent within each state and year.

By following this methodology, the daily weather data is transformed into an annual summary by state, making it directly comparable with the annual, state-level migraine data for further analysis.

In [ ]:
# Group by 'year' and 'state', then aggregate the numerical columns
annual_usa_weather_by_stateCity = usa_weather.groupby(['year', 'state', 'city_name']).agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'sum',
    'snow_depth_mm': 'sum',
    'avg_wind_dir_deg': 'mean',
    'avg_wind_speed_kmh': 'mean',
    'peak_wind_gust_kmh': 'mean',
    'avg_sea_level_pres_hpa': 'mean',
    'latitude': 'first',  # Assuming all latitudes are the same for a given year and state
    'longitude': 'first'  # Assuming all longitudes are the same for a given year and state
}).reset_index()

# Review the shape of the new dataframe
annual_usa_weather_by_stateCity.shape

In [ ]:
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.1.3 Drop Wind Gust and Wind Direction Columns

In [ ]:
annual_usa_weather_by_stateCity.drop(columns=['peak_wind_gust_kmh', 'avg_wind_dir_deg'], inplace=True)

annual_usa_weather_by_stateCity.shape

In [ ]:
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.1.4 Linear Interpolation for Average Wind Speed

In [ ]:
# Handle missing values for the `avg_wind_speed_kmh` column utilizing linear interpolation
annual_usa_weather_by_stateCity['avg_wind_speed_kmh'].interpolate(method='linear', inplace=True)

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
# Check the shape of the dataframe
annual_usa_weather_by_stateCity.shape

#### 5.4.2 Pressure-related Columns

Sea level pressure can vary greatly depending on the location of the city and the main focus of this analysisis is to see if there is any correlation between sudden changes in sea level pressure and migraines.  As a result, we will not fill in missing values for the `avg_sea_level_pres_hpa` column without further research.  We will work through four different scenarios to determine which seems most accurate for this situation.
- Scenario #1: Leave/drop missing values for the `avg_sea_level_pres_hpa` value    
- Scenario #2: Utilize linear interpolation to fill in missing values for the `avg_sea_level_pres_hpa` column
- Scenario #3: Utilize forward fill to fill in missing values for the `avg_sea_level_pres_hpa` column
- Scenario #4: Utilize backward fill to fill in missing values for the `avg_sea_level_pres_hpa` column

##### 5.4.2.1 Leaving/Dropping Missing Values (Scenario #1)

In [ ]:
# Descriptive statistics for the `avg_sea_level_pres_hpa` column
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].describe()

##### 5.4.2.2 Linear Interpolation (Scenario #2)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing linear interpolation
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].interpolate(method='linear')

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.2.3 Forward Fill (Scenario #3)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing forward fill
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].ffill()

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.2.4 Backward Fill (Scenario #4)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing backward fill
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].bfill()

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
annual_usa_weather_by_stateCity.describe()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Define common x and y limits
x_limits = [1005, 1025]  # Replace with the min and max values across all datasets for the x-axis
y_limits = [0, 125]  # Replace with the max frequency across all datasets for the y-axis

# Calculate number of bins for each dataset using the Square Root Rule
num_bins1 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].dropna())))
num_bins2 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'].dropna())))
num_bins3 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'].dropna())))
num_bins4 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'].dropna())))

# Plot each histogram on a different subplot
axes[0, 0].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].dropna(), bins=num_bins1, color='blue')
axes[0, 0].set_title('Original')
axes[0, 0].set_xlim(x_limits)
axes[0, 0].set_ylim(y_limits)

axes[0, 1].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'].dropna(), bins=num_bins2, color='green')
axes[0, 1].set_title('Linear Interpolated')
axes[0, 1].set_xlim(x_limits)
axes[0, 1].set_ylim(y_limits)

axes[1, 0].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'].dropna(), bins=num_bins3, color='red')
axes[1, 0].set_title('Forward Fill')
axes[1, 0].set_xlim(x_limits)
axes[1, 0].set_ylim(y_limits)

axes[1, 1].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'].dropna(), bins=num_bins4, color='purple')
axes[1, 1].set_title('Backward Fill')
axes[1, 1].set_xlim(x_limits)
axes[1, 1].set_ylim(y_limits)

for ax in axes.flat:
    ax.set(xlabel='Sea Level Pressure (hPa)', ylabel='Frequency')
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

# Display all subplots
plt.tight_layout()
plt.show()


##### 5.4.2.5 Decision on Filling Missing Values

After a thorough review of all four scenarios, we've decided to employ **Scenario #2 (linear interpolation)** for filling the missing values in the `avg_sea_level_pres_hpa` column. The rationale behind this choice is manifold:

- **Representation of Data**: Linear interpolation provides a smoother distribution than the other methods. This approach does not heavily skew the tail ends of the distribution, ensuring a more natural representation.
  
- **Preservation of Original Distribution**: Linear interpolation appears to retain the original data distribution more faithfully when filling in missing values, without introducing any discernible bias towards specific values.

- **Percentage of Missing Values**: With only 138 missing values, which represents 9.47% of the total count of 1457, the sheer accuracy of the method is not as paramount as it would be with a more substantial portion of missing values. Nevertheless, it's essential to utilize a method that delivers reliability, and linear interpolation does just that.

**Analysis of Alternative Methods:**

- The **forward-fill method**, though commendable, might introduce bias as it overlooks subsequent values after a missing point. It stands as our second preference.
  
- The **backward-fill method** is our third choice. While it does consider subsequent data points, its accuracy seems to trail the forward-fill method.
  
- Lastly, simply **leaving or dropping missing values** is the least appealing choice, as it disregards the rest of the dataset's information.


##### 5.4.2.6 Linear Interpolation for Average Sea Level Pressure

In [ ]:
# copy the dataframe
annual_usa_weather_by_stateCity = annual_usa_weather_by_stateCity.copy()

# Fill missing values in the `avg_sea_level_pres_hpa` column with the linear values
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear']

# Drop the `avg_sea_level_pres_hpa_ffill` and `avg_sea_level_pres_hpa_bfill` columns
annual_usa_weather_by_stateCity.drop(columns=['avg_sea_level_pres_hpa_ffill', 'avg_sea_level_pres_hpa_bfill'], inplace=True)

# check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
# Filter out rows where either of the two columns is NaN
filtered_df = annual_usa_weather_by_stateCity.dropna(subset=['avg_sea_level_pres_hpa', 'avg_sea_level_pres_hpa_linear'])

# Calculate mean absolute error
mae = mean_absolute_error(filtered_df['avg_sea_level_pres_hpa'], filtered_df['avg_sea_level_pres_hpa_linear'])

# Print the mean absolute error
print(f"Mean Absolute Error: {mae}")

In [ ]:
annual_usa_weather_by_stateCity.drop(columns=['avg_sea_level_pres_hpa_linear'], inplace=True)

annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
annual_usa_weather_by_stateCity.head()

In [ ]:
annual_usa_weather_by_stateCity.shape

### 5.6 Join Migraine Data with Weather Data

Now that the weather data has been aggregated to match the migraine data, we can join the two datasets together. These datasets will be joined on the `year` and `state` columns from both the USA weather data and the migraine data.

In [ ]:
filtered_migraine_data.head()

In [ ]:
filtered_migraine_data.shape

In [ ]:
# Combine USA combined weather with migraine data
usa_weather_migraine = annual_usa_weather_by_stateCity.merge(filtered_migraine_data, 
                                      how='left', 
                                      left_on=['year', 'state'], 
                                      right_on=['year', 'state']
                                      )

# Review the shape of the new dataframe
usa_weather_migraine.shape

## 6. Feature Engineering

Discussing any new features that were created and why they were created. Also, discuss any features that were dropped and why they were dropped.

### 6.1 New Features

#### 6.1.1 Converting Celsius to Fahrenheit

##### 6.1.1.1 Convert `avg_temp_c` to `avg_temp_f`

In [ ]:
from temp_conversion import celsius_to_fahrenheit

usa_weather_migraine['avg_temp_f'] = usa_weather_migraine['avg_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.2 Convert `min_temp_c` to `min_temp_f`

In [ ]:
usa_weather_migraine['min_temp_f'] = usa_weather_migraine['min_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.3 Convert `max_temp_c` to `max_temp_f`

In [ ]:
usa_weather_migraine['max_temp_f'] = usa_weather_migraine['max_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.4 Reorder Temperature Columns

In [ ]:
# Reorder temperature columns
temp_col = usa_weather_migraine.pop('avg_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(3, 'avg_temp_f', temp_col)

In [ ]:
# Reorder temperature columns
temp_col1 = usa_weather_migraine.pop('min_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(4, 'min_temp_f', temp_col1)

In [ ]:
# Reorder temperature columns
temp_col2 = usa_weather_migraine.pop('max_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(5, 'max_temp_f', temp_col2)

In [ ]:
usa_weather_migraine

##### 6.1.1.5 Convert `precipitation_mm` to `precipitation_in`

In [ ]:
from temp_conversion import mM_to_inches

usa_weather_migraine['precipitation_in'] = usa_weather_migraine['precipitation_mm'].apply(mM_to_inches)


In [ ]:
# Reorder temperature columns
temp_col3 = usa_weather_migraine.pop('precipitation_in')

# Insert columns at new position
usa_weather_migraine.insert(9, 'precipitation_in', temp_col3)

In [ ]:
usa_weather_migraine.head()

#### 6.1.2 Sea Level Pressure Missing Values

- `avg_sea_level_pres_hpa`: original column with missing values
- `avg_sea_level_pres_hpa_linear`: calculated by using linear interpolation to fill in missing values for the average sea level pressure
- `avg_sea_level_pres_hpa_ffill`: calculated by using forward fill to fill in missing values for the average sea level pressure
- `avg_sea_level_pres_hpa_bfill`: calculated by using backward fill to fill in missing values for the average sea level pressure

### 6.2 Dropped Features

In [ ]:
# usa_weather_migraine.drop(columns=['avg_temp_c', 'min_temp_c', 'max_temp_c'], inplace=True)

# usa_weather_migraine

In [ ]:
# usa_weather_migraine.drop(columns='precipitation_mm', inplace=True)

# usa_weather_migraine

## 7. Summary

Summarize the data preprocessing steps that were taken in this notebook.

## 8. Next Steps

Discuss any next steps that should be taken in the data analysis process/modeling phases.